In [90]:
"""
------------------------------------------------------------------------------
About
In this data analysis, the proportion of mission calls (fire/ems) 
that reach the 10-minute response time is calculated for each admin area.

The sum of mission_count_ems, mission_count_fire and mission_count_technical_rescue 
was taken as the total sum of mission calls.
------------------------------------------------------------------------------
"""

'\n------------------------------------------------------------------------------\nAbout\nIn this data analysis, the proportion of mission calls (fire/ems) \nthat reach the 10-minute response time is calculated for each admin area.\n\nThe sum of mission_count_ems, mission_count_fire and mission_count_technical_rescue \nwas taken as the total sum of mission calls.\n------------------------------------------------------------------------------\n'

In [91]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

In [92]:
"""
------------------------------------------------------------------------------
Import data
------------------------------------------------------------------------------
"""

df = pd.read_csv("../data/raw/wyx48.csv", dtype={"planning_room_id":str})

df_pgr = pd.read_csv("../data/raw/PGR.csv", dtype={"PGR_ID": str})

df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,mission_count_ems_critical_timegoal_reached,mission_count_fire_timegoal_reached,response_time_ems_critical_mean,response_time_ems_critical_median,response_time_ems_critical_std,response_time_ems_critical_cpr_mean,response_time_ems_critical_cpr_median,response_time_ems_critical_cpr_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_median,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std
0,1100101,Stülerstraße,426,293,216,3,33,9,182,21,110,19,593.758242,572.5,179.061417,423.333333,415.0,22.678918,556.222222,547.0,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826
1,1100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,230,49,616.063927,588.5,209.523958,539.250000,527.0,103.316262,637.393258,618.0,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133
2,1100103,Lützowstraße,697,534,373,6,36,29,331,10,200,9,589.604230,552.0,207.859285,472.500000,475.5,33.786092,588.366667,573.0,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947
3,1100104,Körnerstraße,294,255,177,2,21,18,142,2,83,2,570.260563,540.0,196.087536,442.500000,442.5,20.506097,623.352941,637.0,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610
4,1100205,Wilhelmstraße,581,500,332,5,51,30,276,23,151,18,611.612319,582.5,201.953546,487.750000,482.5,93.749222,643.435897,653.0,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975


In [93]:

df_pgr.rename(columns={"PGR_ID": "pgr_id"}, inplace=True)

df_pgr.head()


,pgr_id,PGR_NAME,BEZ,STAND,GROESSE_M2
0,0930,Treptow-Köpenick 3,9,01.01.2021,4.068962e+07
1,0760,Marienfelde / Lichtenrade,7,01.01.2021,1.919230e+07
2,0920,Treptow-Köpenick 2,9,01.01.2021,1.920230e+07
3,0220,Kreuzberg Süd,2,01.01.2021,4.746238e+06
4,1260,Märkisches Viertel,12,01.01.2021,4.783336e+06


In [94]:
#add missing 0 in planning_room_id (LOR)
df["planning_room_id"] = df["planning_room_id"].str.zfill(8)
df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,mission_count_ems_critical_timegoal_reached,mission_count_fire_timegoal_reached,response_time_ems_critical_mean,response_time_ems_critical_median,response_time_ems_critical_std,response_time_ems_critical_cpr_mean,response_time_ems_critical_cpr_median,response_time_ems_critical_cpr_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_median,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std
0,01100101,Stülerstraße,426,293,216,3,33,9,182,21,110,19,593.758242,572.5,179.061417,423.333333,415.0,22.678918,556.222222,547.0,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826
1,01100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,230,49,616.063927,588.5,209.523958,539.250000,527.0,103.316262,637.393258,618.0,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133
2,01100103,Lützowstraße,697,534,373,6,36,29,331,10,200,9,589.604230,552.0,207.859285,472.500000,475.5,33.786092,588.366667,573.0,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947
3,01100104,Körnerstraße,294,255,177,2,21,18,142,2,83,2,570.260563,540.0,196.087536,442.500000,442.5,20.506097,623.352941,637.0,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610
4,01100205,Wilhelmstraße,581,500,332,5,51,30,276,23,151,18,611.612319,582.5,201.953546,487.750000,482.5,93.749222,643.435897,653.0,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975


In [95]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

df["pgr_id"] = df["planning_room_id"].str[:4]

df.head()

,planning_room_id,planning_room_name,mission_count_all,mission_count_ems,mission_count_ems_critical,mission_count_ems_critical_cpr,mission_count_fire,mission_count_technical_rescue,mission_count_ems_critical_timegoal_computed,mission_count_fire_timegoal_computed,mission_count_ems_critical_timegoal_reached,mission_count_fire_timegoal_reached,response_time_ems_critical_mean,response_time_ems_critical_median,response_time_ems_critical_std,response_time_ems_critical_cpr_mean,response_time_ems_critical_cpr_median,response_time_ems_critical_cpr_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_median,response_time_fire_time_to_first_pump_std,response_time_fire_time_to_first_ladder_mean,response_time_fire_time_to_first_ladder_median,response_time_fire_time_to_first_ladder_std,response_time_fire_time_to_full_crew_mean,response_time_fire_time_to_full_crew_median,response_time_fire_time_to_full_crew_std,response_time_technical_rescue_mean,response_time_technical_rescue_median,response_time_technical_rescue_std,pgr_id
0,01100101,Stülerstraße,426,293,216,3,33,9,182,21,110,19,593.758242,572.5,179.061417,423.333333,415.0,22.678918,556.222222,547.0,110.888486,693.518519,672.0,131.405529,744.571429,709.0,129.539018,921.625000,937.0,256.244826,0110
1,01100102,Großer Tiergarten,1065,887,520,6,110,54,438,63,230,49,616.063927,588.5,209.523958,539.250000,527.0,103.316262,637.393258,618.0,136.057777,732.686747,719.0,140.667999,843.904762,802.0,180.813459,804.857143,805.5,306.051133,0110
2,01100103,Lützowstraße,697,534,373,6,36,29,331,10,200,9,589.604230,552.0,207.859285,472.500000,475.5,33.786092,588.366667,573.0,153.350058,654.964286,662.0,108.650811,734.000000,734.0,105.222516,803.625000,794.5,318.006947,0110
3,01100104,Körnerstraße,294,255,177,2,21,18,142,2,83,2,570.260563,540.0,196.087536,442.500000,442.5,20.506097,623.352941,637.0,56.869303,736.555556,726.0,106.210237,696.500000,696.5,37.476659,972.062500,980.5,152.431610,0110
4,01100205,Wilhelmstraße,581,500,332,5,51,30,276,23,151,18,611.612319,582.5,201.953546,487.750000,482.5,93.749222,643.435897,653.0,91.957311,750.682927,740.0,113.773995,807.826087,800.0,96.747399,912.523810,855.0,329.384975,0110


In [97]:
#merge orgiginal data frame with admin area data frame
df_merged= pd.merge(df, df_pgr, on= "pgr_id", how="left")

In [98]:
"""
------------------------------------------------------------------------------
response_time_ems_critical_mean (in seconds)
response_time_ems_critical_std (in seconds)

response_time_fire_time_to_first_pump_mean (in seconds)
response_time_fire_time_to_first_pump_std (in seconds)

planning_room_name
------------------------------------------------------------------------------
"""
#subset
df_TimeEms = df_merged[["pgr_id", "planning_room_name", "PGR_NAME", "response_time_ems_critical_mean", "response_time_ems_critical_std", "response_time_fire_time_to_first_pump_mean", "response_time_fire_time_to_first_pump_std"]]
df_TimeEms.head()

,pgr_id,planning_room_name,PGR_NAME,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
0,0110,Stülerstraße,Zentrum,593.758242,179.061417,556.222222,110.888486
1,0110,Großer Tiergarten,Zentrum,616.063927,209.523958,637.393258,136.057777
2,0110,Lützowstraße,Zentrum,589.604230,207.859285,588.366667,153.350058
3,0110,Körnerstraße,Zentrum,570.260563,196.087536,623.352941,56.869303
4,0110,Wilhelmstraße,Zentrum,611.612319,201.953546,643.435897,91.957311


In [99]:
# pd.set_option("display.max_colwidth", None)
# pd.set_option("display.width", 1000)
# pd.set_option("display.max_columns", None)


In [100]:
#response time ems
df_TimeEms["response_time_ems_critical_mean"] = (df_TimeEms["response_time_ems_critical_mean"] /60).round(2)
df_TimeEms["response_time_ems_critical_std"] = (df_TimeEms["response_time_ems_critical_std"] /60).round(2)

#response time fire time to first pump
df_TimeEms["response_time_fire_time_to_first_pump_mean"] = (df_TimeEms["response_time_fire_time_to_first_pump_mean"] /60).round(2)
df_TimeEms["response_time_fire_time_to_first_pump_std"] = (df_TimeEms["response_time_fire_time_to_first_pump_std"] /60).round(2)

df_TimeEms

/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_22834/1769925805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TimeEms["response_time_ems_critical_mean"] = (df_TimeEms["response_time_ems_critical_mean"] /60).round(2)
/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_22834/1769925805.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TimeEms["response_time_ems_critical_std"] = (df_TimeEms["response_time_ems_critical_std"] /60).round(2)
/var/folders/_f/022l68353h9c2pr_3t55ns1m

,pgr_id,planning_room_name,PGR_NAME,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
0,0110,Stülerstraße,Zentrum,9.90,2.98,9.27,1.85
1,0110,Großer Tiergarten,Zentrum,10.27,3.49,10.62,2.27
2,0110,Lützowstraße,Zentrum,9.83,3.46,9.81,2.56
3,0110,Körnerstraße,Zentrum,9.50,3.27,10.39,0.95
4,0110,Wilhelmstraße,Zentrum,10.19,3.37,10.72,1.53
...,...,...,...,...,...,...,...
537,1260,Rollbergesiedlung,Märkisches Viertel,11.13,2.99,11.71,2.06
538,1260,Treuenbrietzener Straße,Märkisches Viertel,12.20,3.49,13.07,4.54
539,1260,Märkisches Zentrum,Märkisches Viertel,12.31,3.81,12.48,4.21
540,1260,Dannenwalder Weg,Märkisches Viertel,11.60,3.83,9.53,3.46


In [101]:
#Test example Arkonaplatz
df_TimeEms[df_TimeEms["planning_room_name"] == "Arkonaplatz"]

,pgr_id,planning_room_name,PGR_NAME,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
15,0110,Arkonaplatz,Zentrum,9.24,3.42,8.92,1.43


In [102]:
#sort descending
df_TimeEms.sort_values(by= "response_time_ems_critical_mean", ascending=False, inplace=True)
df_TimeEms

/var/folders/_f/022l68353h9c2pr_3t55ns1m0000gn/T/ipykernel_22834/1685112705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_TimeEms.sort_values(by= "response_time_ems_critical_mean", ascending=False, inplace=True)


,pgr_id,planning_room_name,PGR_NAME,response_time_ems_critical_mean,response_time_ems_critical_std,response_time_fire_time_to_first_pump_mean,response_time_fire_time_to_first_pump_std
408,0930,Schmöckwitz/Rauchfangswerder,Treptow-Köpenick 3,17.36,4.71,13.66,4.03
169,0440,Güterbahnhof Grunewald,Wilmersdorf Süd,15.04,6.11,NaN,NaN
415,0940,Müggelheim,Treptow-Köpenick 4,14.91,4.44,14.53,5.11
407,0930,Karolinenhof,Treptow-Köpenick 3,14.69,4.00,11.23,1.51
99,0330,Blankenburger Süden,Nördliches Weißensee,14.57,NaN,11.02,NaN
...,...,...,...,...,...,...,...
296,0720,Feurigstraße,Schöneberg Süd,8.54,3.61,7.49,2.88
62,0230,Oranienplatz,Kreuzberg Ost,8.51,2.60,8.39,1.74
191,0450,Leon-Jessel-Platz,Wilmersdorf Zentrum,8.43,2.49,9.31,3.75
211,0510,Kurstraße,Spandau Mitte / Nord,8.32,3.34,9.20,2.65


In [103]:
df_TimeEms.to_csv("../data/csv/responseTimeEMS.csv", index=False)